In [ ]:
import numpy as np 
import time
from IPython.display import clear_output

from six.moves.urllib.request import urlopen
from contextlib import closing
import json
import k3d

plot = k3d.plot()
plot.display()

In [ ]:
%%time 
chunk_count = 3
chunk_count = max(min(chunk_count, 90),0)

for i in range(chunk_count):
    print("retrieving chunk: ", i)
    with closing(urlopen("http://archive.cyark.org/3d/models/tikal9m-" + str(i) + ".js")) as response:
        %time chunk = json.loads(response.read().decode('utf8'))
        positions = []
        colors = []

        for point in chunk:
            positions.append(point[0:3])
            colors.append(int(point[4], 16))
            
        plot += k3d.points(positions, colors, point_size=0.5, shader="3d") 
        clear_output(wait=True)

        print("Points count:", len(colors),"ith:",i)
        if i>0:
            plot.camera_auto_fit = False
            plot.grid_auto_fit = False
print("Total points:",sum([p.positions.shape[0] for p in plot.objects])     )

In [ ]:
for p in plot.objects:
    p.shader = '3d'
    p.point_size = 2.71

In [ ]:
sigma=10.0
beta=8./3
rho=28.0
def lorenz_deriv(X, sigma=sigma, beta=beta, rho=rho):
    """Compute the time-derivative of a Lorenz system."""
    x, y, z = X.T
    return np.vstack([sigma * (y - x), x * (rho - z) - y, x * y - beta * z]).T

In [ ]:
for p in plot.objects:
    X = p.positions
    for i in range(150):
        X = X + lorenz_deriv(X, sigma=sigma, beta=beta, rho=rho)*0.002
        if i%15==0 and i>0:
            p.positions = X[::1,:]
        #time.sleep(0.1)
        clear_output(wait=True)
        print("iteration:",i)
    p.positions = X

In [ ]:
for i in range(15):
    for p in plot.objects:
        X = p.positions
        for j in range(15):
            X = X + lorenz_deriv(X, sigma=sigma, beta=beta, rho=rho)*0.001
        p.positions = X[:,:]
        clear_output(wait=True)
        print("iteration:",i)
    

In [ ]:
p.point_size = .65